Download dataset from https://childes.talkbank.org/data-xml/

In [ ]:
# https://www.nltk.org/_modules/nltk/corpus/reader/childes.html

In [2]:
from childes_mi.utils.paths import DATA_DIR, ensure_dir, CHILDES_DIR

In [3]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
import urllib.request
from tqdm.autonotebook import tqdm
import zipfile

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### get URLs to download

In [5]:
def isDirectory(url):
    if(url.endswith('/')):
        return True
    else:
        return False

def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    links = []
    for link in maybe_directories:
        #print(link)
        if(isDirectory(link['href'])):
            if link['href'] == '/': continue
            newUrl = url + link['href']  
            #print('newurl', newUrl)
            links.append(findLinks(newUrl)) #recursion happening here
        else:
            if(link['href'].endswith('.zip')):
                print("GOTCHA!", url + link['href']) #now safe and download
                links.append(url + link['href'])
    return links

In [6]:
startUrl = "https://childes.talkbank.org/data-xml/"
urls = findLinks(startUrl)

GOTCHA! https://childes.talkbank.org/data-xml/Biling/Amsterdam.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Bailleul.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/BiSLI.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Biro.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/CHCC.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/CUHK.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/DeHouwer.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/DeHouwerBornstein-protect/DeHouwerBornstein.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Deuchar.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/FerFuLice.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/GNP.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Gelman.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Guthrie.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Hacohen.zip
GOTCHA! https://childes.talkbank.org/data-xml/Biling/Hayashi.zip

GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Belfast.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Cruttenden.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Fletcher.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Forrester.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Gathburn.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Howe.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Korman.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Lara.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/MPI-EVA-Manchester.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Manchester.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Nuffield.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Thomas.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Tommerdahl.zip
GOTCHA! https://childes.talkbank.org/data-xml/Eng-UK/Wells.zip
GOTCHA! https://childes.talkbank.org/data-xml/French/Champaud.zip
GOTCHA! h

GOTCHA! https://childes.talkbank.org/data-xml/Scandinavian/Icelandic/Einarsdottir.zip
GOTCHA! https://childes.talkbank.org/data-xml/Scandinavian/Icelandic/Kari.zip
GOTCHA! https://childes.talkbank.org/data-xml/Scandinavian/Norwegian/Ringstad.zip
GOTCHA! https://childes.talkbank.org/data-xml/Scandinavian/Swedish/Andren.zip
GOTCHA! https://childes.talkbank.org/data-xml/Scandinavian/Swedish/Lund.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Croatian/Kovacevic.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Czech/Chroma.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Polish/CDS.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Polish/Szuman.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Russian/Protassova.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Russian/Tanja.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Serbian/SCECL.zip
GOTCHA! https://childes.talkbank.org/data-xml/Slavic/Slovenian/Zagar.zip
GOTCHA! https://childes.

In [7]:
# flatten the recursive directiories
flatten = lambda l: sum(map(flatten, l), []) if isinstance(l, list) else [l]
urls = flatten(urls)
urls[:3]

['https://childes.talkbank.org/data-xml/Biling/Amsterdam.zip',
 'https://childes.talkbank.org/data-xml/Biling/Bailleul.zip',
 'https://childes.talkbank.org/data-xml/Biling/BiSLI.zip']

### download from URLs

In [8]:

output_directory = CHILDES_DIR
for url in tqdm(urls):
    # get save location
    save_loc = output_directory / url[38:]
    # make sure there is a directory to save to
    ensure_dir(save_loc)
    # save to disk
    urllib.request.urlretrieve(url, save_loc)

### unzip files

In [9]:
zip_files = list(CHILDES_DIR.glob('**/*.zip'))
zip_files[:3]

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/NLTK_Data_Dir/corpora/Scandinavian/Danish/Plunkett.zip'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/NLTK_Data_Dir/corpora/Scandinavian/Swedish/Lund.zip'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/NLTK_Data_Dir/corpora/Scandinavian/Swedish/Andren.zip')]

In [10]:
for zf in tqdm(zip_files):
    with zipfile.ZipFile(zf, 'r') as zip_ref:
        zip_ref.extractall(zf.parent)